In [1]:
import tensorflow as tf
from tensorflow.contrib import eager
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import keras

In [2]:
tf.enable_eager_execution()

In [3]:
data = np.load("./adult/adult_train.npz")

In [19]:
X = data.f.x
y = data.f.y
A = data.f.a
X.shape

(32561, 113)

In [20]:
columns_file = open('./adult/adult_headers.txt','r')
columns = columns_file.read()
columns_names = columns.split("\n")

In [21]:
X = pd.DataFrame(X,columns=columns_names) 
y = pd.DataFrame(y,columns=['income'])
A = pd.DataFrame(A,columns=['A'])
dataset = pd.concat([X,y,A],axis=1)

In [22]:
dataset[:3]

,age_u20,age_u30,age_u40,age_u50,age_u60,age_u70,age_u80,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,...,country_Thailand,country_Yugoslavia,country_El-Salvador,country_Trinadad&Tobago,country_Peru,country_Hong,country_Holand-Netherlands,country_?,income,A
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


1- Name the 10 features which are most correlated with Y, and the 10 which are most correlated with A, as measured by (absolute) Pearson correlation (ignore any NaN correlations you see.

In [23]:
data_corr_income = dataset.corr()['income']
data_corr_A = dataset.corr()['A']

In [24]:
answer1 = data_corr_income.sort_values(ascending=False)[1:11]
answer2= data_corr_A.sort_values(ascending=False)[1:11]

In [25]:
print(answer1)
print("#####################################################")
print(answer2)

marital-status_Married-civ-spouse    0.444696
relationship_Husband                 0.401035
education_num                        0.335154
hours-per-week                       0.229689
capital-gain                         0.223329
sex_Male                             0.215980
A                                    0.215980
occupation_Exec-managerial           0.214861
occupation_Prof-specialty            0.185866
education_Bachelors                  0.180485
Name: income, dtype: float64
#####################################################
sex_Male                             1.000000
relationship_Husband                 0.580135
marital-status_Married-civ-spouse    0.431805
hours-per-week                       0.229309
occupation_Craft-repair              0.223128
income                               0.215980
occupation_Transport-moving          0.132468
workclass_Self-emp-not-inc           0.107451
race_White                           0.103486
occupation_Farming-fishing           0.1000

In [26]:
X_train = tf.convert_to_tensor(X.to_numpy(),dtype=tf.float32)
y_train = tf.convert_to_tensor(y.to_numpy(),dtype=tf.float32)

In [27]:
m,n = X.shape

# Modeling

In [28]:
classifier = tf.keras.Sequential(
    [tf.keras.layers.Dense(input_shape=(n,),units=200,activation=tf.nn.relu),
    tf.keras.layers.Dense(units = 300,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=50,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=20,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1)]
)

In [32]:
def loss(y,y_):
    mloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=y_))
    return mloss


def gradient(model,x,y):
    with tf.GradientTape() as tape:
        y_ = model(x)
        mloss = loss(y,y_)
        return mloss, tape.gradient(mloss,model.trainable_variables)
    
def accuracy(y,y_):
    y_ = tf.nn.sigmoid(y_)
    ans = tf.cast((y_>=0.5),dtype= tf.float32)
    res = tf.cast(tf.equal(ans,y),tf.float32)
    
    return tf.reduce_mean(res)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
global_step = tf.Variable(0)


In [33]:
epochs = 500
for ep in range(epochs):
    print("Epoch: ", ep)
    y_ = classifier(X_train)
    mloss, grad = gradient(classifier,X_train,y_train)
    optimizer.apply_gradients(zip(grad, classifier.trainable_variables),global_step=global_step)
    
    print("Loss: ",mloss.numpy(), "######### "," Accuracy: ", accuracy(y_train,y_).numpy())


Epoch:  0
Loss:  3.8204904 #########   Accuracy:  0.7756519
Epoch:  1
Loss:  300.10458 #########   Accuracy:  0.75919044
Epoch:  2
Loss:  76.60125 #########   Accuracy:  0.76001966
Epoch:  3
Loss:  14.3391905 #########   Accuracy:  0.76001966
Epoch:  4
Loss:  1.7395462 #########   Accuracy:  0.7789073
Epoch:  5
Loss:  1.838211 #########   Accuracy:  0.7789073
Epoch:  6
Loss:  2.2259562 #########   Accuracy:  0.7797365
Epoch:  7
Loss:  2.3061419 #########   Accuracy:  0.7797365
Epoch:  8
Loss:  0.8978553 #########   Accuracy:  0.7810571
Epoch:  9
Loss:  38.237816 #########   Accuracy:  0.75919044
Epoch:  10
Loss:  2.666266 #########   Accuracy:  0.7789073
Epoch:  11
Loss:  3.8449075 #########   Accuracy:  0.7789073
Epoch:  12
Loss:  4.7512865 #########   Accuracy:  0.7797365
Epoch:  13
Loss:  5.5746374 #########   Accuracy:  0.7797365
Epoch:  14
Loss:  5.5526175 #########   Accuracy:  0.7805043
Epoch:  15
Loss:  5.877717 #########   Accuracy:  0.7789073
Epoch:  16
Loss:  5.251256 ######